In [ ]:
import numpy as np

In [ ]:
def S(X):
    return X**3

class Population:
    
    def __init__(self, M, N=200, g=1.5, tau=0.030, dt=1e-3, bias=4, 
                 sta_dt=0.005, eta=0.5, alpha_trace=0.33, perturb_rate=3.0):
        self.M, self.N = M, N
        self.g   = g
        self.tau = tau
        self.t, self.dt, self.sta_dt = 0, dt, sta_dt
        self.eta, self.alpha_trace = eta, alpha_trace
        self.perturb_rate = perturb_rate
        self.bias = np.random.choice(N, size=5, replace=False)
        self._ouput = np.random.choice(N, size=1, replace=False)[0] # output neuro
        self.bias = self.bias[:-1]
        
        self.B = np.random.uniform(low=-1, high=1, size=(N, M))
        self.J = np.random.normal(loc=0.0, scale=self.g**2/self.N, size=(N, N))
        self.R_expected = {} # history of rewards by trial type
        self.reset()

    def reset(self):
        self.X = np.random.uniform(low=-0.1, high=0.1, size=(self.N, 1))
        self.e = np.zeros((self.N, self.N))
        
        self._sta_X = [self.X] 
        self.t = 0
        
    def output(self):
        return np.tanh(self.X[self._ouput][0])
        
    def sta_X(self):
        """Short-term average of X"""
        self._sta_X = self._sta_X[-int(self.sta_dt/self.dt):]
        return np.mean(self._sta_X, axis=0)
        
    def step(self, U):
        """Advance time by self.dt"""
        self.X[self.bias] = 1.0
        perturb_flags = np.random.random((self.N, 1)) < (self.dt*self.perturb_rate)
        perturb = perturb_flags * np.random.uniform(low=-0.5, high=0.5, size=(self.N, 1))
            
        A = np.tanh(self.X) # activation a t-1                                                           # Equation 2
        self.X += self.dt/self.tau*(-self.X + np.dot(self.J, A) + np.dot(self.B, U) + perturb)           # Equation 1
        self.X = np.clip(self.X, -1.0, 1.0) # necessary ?
        self.t += self.dt
        
        self._sta_X.append(self.X)
        self.e += np.dot(A, S(self.X - self.sta_X()).T)                                                  # Equation 3
        
    def end_trial(self, inputs, R):
        inputs = tuple(tuple(map(tuple, e)) for e in inputs) # hashable inputs
        if not inputs in self.R_expected:
            self.R_expected[inputs] = 0.0
        self.R_expected[inputs] = self.alpha_trace*self.R_expected[inputs] + (1-self.alpha_trace)*R      # Equation 5
        delta_J = self.eta*self.e*(R - self.R_expected[inputs])                                          # Equation 4
        print('delta_J', np.mean(delta_J))

        self.J += np.clip(delta_J, -1e-4, 1e-4)

In [ ]:
def trial(pop, U_1, U_2):
    U_zero = np.zeros((pop.M, 1))
    outputs = []
    
    while pop.t <= 0.200:
        pop.step(U_1)
    while pop.t <= 0.400:
        pop.step(U_zero)
    while pop.t <= 0.600:
        pop.step(U_2)
    while pop.t <= 0.700:
        pop.step(U_zero)
    while pop.t <= 1.0:
        pop.step(U_zero)
        outputs.append(pop.output())

    target = 1.0
    if np.all(U_1 == U_2):
        target = -1.0
    
    R = 0
    if np.mean(outputs)*target > 0:
        R = 1
    
    
#     if (np.all(U_1 == U_2)):
#         R = (np.mean(outputs)>0.5)*1.
#     else:
#         R = (np.mean(outputs)<-0.5)*1.
    error = np.abs(target - np.mean(outputs)) # computing reward
    pop.end_trial([U_1, U_2], error)
    
    return R, outputs, error

In [ ]:
np.random.seed(0)

U_A    = np.array([[1], [0]])
U_B    = np.array([[0], [1]])

pop = Population(2, N=200, perturb_rate=50.0)
for i in range(10000): # takes a long time.
    idx1, idx2 = np.random.choice(2), np.random.choice(2)
    U_1 = [U_A, U_B][idx1]
    U_2 = [U_A, U_B][idx2]
    R, outputs, error = trial(pop, U_1, U_2)
    
    pop.reset()
    print("{:03d} {}{}: {} {}".format(i, ['A', 'B'][idx1], ['A', 'B'][idx2], R, error))